modelado (paramétrico, no paramétrico y ML) para estimar parámetros de la vegetación a partir de variables SAR

**Objetivo (Y)**: estimar parámetros biofísicos de vegetación medidos en campo (biomasa, contenido de agua, indice de biomasa por altura IBA).

**Predictores (X)**: derivados de SAR Sentinel‑1

# 1- Carga y preparación de datos

In [1]:
# Cargar funcion almacenada en .py
from os import sys
# Añadir la ruta del directorio de funciones al path
functions_dir = "c:/Users/Usuario/OneDrive/20211021_Int_Datafusion/20240118_MSc_MR_Datafusion/Processing/Sentinel/functions"
sys.path.append(functions_dir)
from baseline_linear import *

# 1) Datos
data_path = 'outputs/selected_variables_for_modeling.csv'
df = load_dataset(data_path)
df.head()

,id_point,datetime,VWC,total_biomass,IBA,Sigma0_RATIO_VH_VV,Gamma0_RATIO_VH_VV,PC4,Alpha
0,G1-1,2025-02-19 11:11:00,315.945035,1263.78014,1853.050059,0.809757,0.800494,1.222716,25.555786
1,G1-13,2025-02-19 11:48:00,166.896690,667.58676,755.188643,0.809795,0.800540,1.294960,30.238604
2,G1-17,2025-02-19 11:59:00,152.696690,610.78676,690.935249,0.809814,0.800561,1.309476,28.057308
3,G1-25,2025-02-19 12:17:00,188.196690,752.78676,865.272138,0.809852,0.800572,1.361530,22.238356
4,G1-9,2025-02-19 11:30:00,283.145035,1132.58014,1926.156701,0.809778,0.800519,1.462057,24.861599


<Figure size 640x480 with 0 Axes>

# 2- Model baseline
## Lineal

In [2]:
# Definir las columnas de características y objetivo
target_col = ['VWC', 'total_biomass', 'IBA']
feature_cols = ['Sigma0_RATIO_VH_VV', 'Gamma0_RATIO_VH_VV', 'PC4', 'Alpha']
X, y = select_xy(df, target_col, feature_cols)
num_features = list(X.columns)

# Dividir en conjuntos de prueba y validacion
splits = random_splits(X, y, val_size=0.3, random_state=42)
(X_train, y_train) = splits["train"]
(X_val,   y_val)   = splits["val"]

In [6]:
out_dir = 'outputs/artifacts_linear_baseline'
# Ejecutar el baseline lineal para cada target_col
for target in target_col:
    print(f"Ejecutando LR para: {target}")
    metrics_LR = run_baseline_from_splits(
        X_train, y_train[target],
        X_val, y_val[target],
        target,
        num_features,
        model_type="ridge",
        alpha=1.0,
        outdir=out_dir
    )
    print(f"Resultados para {target}: {metrics_LR}")

Ejecutando LR para: VWC
Resultados para VWC: {'train': {'RMSE': 97.23853675457148, 'MAE': 75.77603384945942, 'R2': 0.24734444898153196, 'rRMSE(%)': 42.646550162941246}, 'val': {'RMSE': 98.56567851142258, 'MAE': 83.30143823038335, 'R2': 0.09409565903495953, 'rRMSE(%)': 48.929878008130196}}
Ejecutando LR para: total_biomass
Resultados para total_biomass: {'train': {'RMSE': 388.9541470182859, 'MAE': 303.1041353978377, 'R2': 0.24734444898153207, 'rRMSE(%)': 42.64655016294124}, 'val': {'RMSE': 394.26271404569036, 'MAE': 333.2057529215335, 'R2': 0.09409565903495942, 'rRMSE(%)': 48.92987800813021}}
Ejecutando LR para: IBA
Resultados para IBA: {'train': {'RMSE': 1406.4941678956945, 'MAE': 952.4627345691537, 'R2': 0.36371793978272837, 'rRMSE(%)': 61.46970245606337}, 'val': {'RMSE': 1088.018326492343, 'MAE': 828.3224805358328, 'R2': -0.18890173871931126, 'rRMSE(%)': 62.4777988254457}}


## No paramétrico

In [ ]:
functions_dir = r"c:/Users/Usuario/OneDrive/20211021_Int_Datafusion/20240118_MSc_MR_Datafusion/Processing/Sentinel/functions"
if functions_dir not in sys.path:
	sys.path.append(functions_dir)

from baseline_nonparam import *
import numpy as np
# Define el ancho de banda
bandwidth_grid = np.logspace(-3, 1, 50) # para total_biomass
out_dir = 'outputs/artifacts_nadaraya_watson_baseline'
out_dir = ensure_outdir(out_dir)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 84-85: truncated \UXXXXXXXX escape (baseline_nonparam.py, line 296)

In [ ]:
# Ejecutar el baseline Nadaraya-Watson para cada target_col
for target in target_col:
    print(f"Ejecutando NW para {target}")
    metrics_nw = run_nw_from_splits(
        X_train, y_train[target],
        X_val, y_val[target],
        target_col=target,
        num_features=num_features,
        bandwidth_grid=bandwidth_grid,
        reg_type="lc",
        outdir=out_dir
    )
    print(f'Metricas Nadaraya-Watson para {target}: {metrics_nw}')

## Random Forest

In [8]:
from baseline_random_forest import *
out_dir = 'outputs/artifacts_random_forest_baseline'

param_dist = {
    "n_estimators": np.arange(100, 1500, 100),  # Puedes probar desde 100 hasta 1500 árboles
    "max_depth": [None, 10, 20, 30, 40, 50],  # Profundidad del árbol
    "min_samples_split": np.arange(2, 21, 2),  # Mínimo de muestras para dividir un nodo
    "min_samples_leaf": np.arange(1, 21, 2),  # Mínimo de muestras para ser hoja
    "max_features": ["auto", "sqrt", "log2"],  # Número de features a probar
    "bootstrap": [True, False]  # Usar muestras con reemplazo o no
}

for target in target_col:
    print(f"Ejecutando RF para {target}")
    metrics_rf = run_rf_from_splits(
        X_train=X_train, 
        y_train=y_train[target],
        X_val=X_val, 
        y_val=y_val[target],
        target_col=target,
        param_dist=param_dist,
        num_features=num_features,
        outdir=out_dir
    )
    print(f'Metricas Random Forest para {target}: {metrics_rf}')

Ejecutando RF para t


KeyError: 't'

## LightGBM

# 3- Optimización de hiperparámetros

# 4- Incertitumbre
Intervalos de predicción:

RF: Quantile Regression Forest (q=0.1/0.9).

LGBM: objetivo quantile o conformal prediction (CV+residuales).

Robustez: bootstrapping por bloque (parcela) y análisis de shift (distribuciones de X entrenamiento vs. test).

Sensibilidad: ablation study eliminando familias de variables (texturas, geometría, ratios) y comparando ΔRMSE.

# 5- Metricas y reporte